In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from tqdm import tqdm

from model import VGG

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import GrapeDataset

In [3]:
def create_annotations(dir_path):
    """ 
    Returns a Dataframe with images and labels
    """
    X = []
    y = []
    i = 0
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            for file in sorted(os.listdir(dir_path + path)):
                if not file.startswith('.'):
                    X.append(path+'/'+file)
                    y.append(i)
            i +=1
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return pd.DataFrame({'filename':X,
                        'label':y})

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
test_path = 'data/test/'
train_path = 'data/train/'    
# Hyperparameters
batch_size = 4
learning_rate = 1e-3
num_epochs = 10
# Loading data
ann_train = create_annotations(train_path)
ann_test = create_annotations(test_path)

transform = transforms.Compose([ transforms.ToTensor(), transforms.Resize((224,224))])
train_data = GrapeDataset(img_dir=train_path, ann_df=ann_train,transform=transform)
test_data = GrapeDataset(img_dir=test_path, ann_df=ann_test,transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Loading the model
model = VGG(num_classes=4)
model.to(device)
# Loss and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

100%|██████████| 5/5 [00:00<00:00, 545.14it/s]


7222 images loaded from data/train/ directory.


100%|██████████| 5/5 [00:00<00:00, 1426.63it/s]


1805 images loaded from data/test/ directory.


In [5]:
X, y = next(iter(train_loader))

In [6]:
device = "mps"
model.to(device)

VGG(
  (conv_stack): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(

In [8]:
num_epochs = 100
for epoch in range(num_epochs):
    X, y = X.to(device), y.to(device)
    preds = model(X)
    loss = loss_fn(preds, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

tensor(0.0203, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0135, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0211, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0212, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0228, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0145, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0208, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0091, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0119, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0152, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0209, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0119, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0128, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0207, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0138, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0162, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0146, device='mps:0', grad_fn=<NllLossBackward0

In [9]:
preds

tensor([[ 4.8792, -1.1897, -2.7335, -2.3931],
        [-1.3759,  5.0562, -2.7271, -3.2692],
        [-1.2180,  4.9299, -2.4953, -2.3726],
        [ 4.6771, -0.8411, -2.3173, -1.6784]], device='mps:0',
       grad_fn=<LinearBackward0>)

In [10]:
y

tensor([0, 1, 1, 0], device='mps:0')

In [11]:
def calculate_accuracy(preds, labels):
    _, predicted = torch.max(preds, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)
    return accuracy

In [12]:
calculate_accuracy(preds, y)

1.0

In [17]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    # Train
    model.train()
    for idx, (X,y) in loop:
        X, y = X.to(device), y.to(device)

        preds = model(X)
        loss = loss_fn(preds, y)
        train_running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_description(f"Epoch:[{epoch}/{num_epochs}](Train)")
        loop.set_postfix(train_loss=loss.item())

    train_loss = train_running_loss / (idx + 1)

    # Val
    val_running_loss = 0.0
    run_accuracy = 0
    model.eval()
    loop = tqdm(enumerate(test_loader), total=len(test_loader), leave=False)
    with torch.no_grad():
        for idx, (X,y) in loop:
            X, y = X.to(device), y.to(device)

            preds = model(X)
            loss = loss_fn(preds, y)
            val_running_loss += loss.item()
            # accuracy = calc_accuracy(preds, mask, cfg.THR)
            # run_accuracy += accuracy.item()
            loop.set_description(f"Epoch:[{epoch}/{num_epochs}](Val)")
            # loop.set_postfix(val_loss=loss.item(), acc=accuracy.item())

KeyboardInterrupt: 